## Name: Karma Tarap
### CSCI E-89C Deep Reinforcement Learning  
### Part II of Assignment 5

# Problem 1 (25 points)

In this problem we consider patients with end-stage liver disease (ESLD). We assume that patient's health condition is fully characterized by the Model for End-stage Liver Disease (MELD) score (Jae-Hyeon Ahn and John Hornberger, Involving patients in the cadaveric kidney transplant allocation process: a decision-theoretic perspective. Manage Sci. 1996;42(5):629–41).

The MELD score ranges from 6 to 40 and is derived based on the probability of survival at 3 months for patients with ESLD. Data in ESLD is usually sparse and often aggregated into Stages. We assume that there are 18 stages based on the ESLD: Stage 1, Stage2, ..., Stage 18. The time step is 1 year and the actions in Stages 1 through 18 are "wait" (denoted by 0) and "transplant" (denoted by 1). 

We assume that the Markov property holds. There are two additional states of the Markov Decision Process: "Posttransplant Life" (denoted by 19) and "Death" (which is denoted by 20 and combines so caled "Pretransplant Death" and "Posttransplant Death"). The only action availible in state "Posttransplant Life" is "wait" and "Death" is the terminal state with no actions. Assume that the length of an episode is T=49, unless it terminates earlier due to the transition to the absorbing state "Death."

We do not know the transition probabilities, but if a patient selects "wait," the possible transitions are   
1) Stage 1->Stage 1, Stage 1->Stage 2, Stage 1->Death  
2) For k in {2,3,4,...17}, Stage k->Stage (k-1), Stage k->Stage k, Stage k->Stage (k+1), Stage k->Death    
3) Stage 18->Stage 17, Stage 18->Stage 18, Stage 18->Death    

If a patient selects "transplant" at Stage k, k=1,2,...,18, the only possible transition is  
4) Stage k->"Posttransplant Life"

Finally, there are two more possible transitions"  
5) "Posttransplant Life"->"Posttransplant Life" and "Posttransplant Life"->"Death"  


The patient gets reward 1 in all states "Stage k" (k=1,2,...,18) and reward 0.2 in the "Posttransplant Life" state - assume that the patient gets these rewards on "exit" from the states, i.e. after we observe the corresponding stage. We assume the discounting parameter $\gamma=0.97$, one of the most common discounting rate used in medical decision making (Gold MR, Siegel JE, Russell LB, Weinstein MC. Cost-Effectiveness in Health and Medicine. Oxford University Press; New York: 1996).


Please consider statistics on 8,000 patients with ESLD saved in the 'ESLD_statistics.csv' file. Each row represents an episode (i.e. one patient) and the columns are the sequences of the patients' states and actions. This data were generated under the behavior policy:

$b(1|k)=0.02$ for $k\in\{1,2,3,4,5,6,7,8,9,10,11,12,13\}$;   
$b(1|14)=0.05$;   
$b(1|15)=0.10$;   
$b(1|16)=0.20$;   
$b(1|17)=0.40$;  
$b(1|18)=0.60$;  

which means that, for example, 5% of paients at stage 14 received a transplant.

   
Please use the Off policy MC control (for estimating $\pi_*$), which corresponds to the weighted importance sampling, to obtain the optimal policy. Please be specific and answer at what stages it is worth considering a transplant and at which stages - not.

In [1]:
import random
from matplotlib import pyplot as plt 
import numpy as np
import pandas as pd
np.set_printoptions(precision=3)

In [2]:
# Read in data and take a peek
elsd = pd.read_csv("ESLD_statistics.csv")
elsd.head()

,S0,A0,S1,A1,S2,A2,S3,A3,S4,A4,...,A45,S46,A46,S47,A47,S48,A48,S49,A49,S50
0,12,0,12,0,13,0,13,0,20,0,...,0,20,0,20,0,20,0,20,0,20
1,3,0,3,0,3,0,3,0,3,0,...,0,20,0,20,0,20,0,20,0,20
2,16,0,16,0,16,1,19,0,19,0,...,0,20,0,20,0,20,0,20,0,20
3,13,0,13,0,13,0,13,0,14,0,...,0,20,0,20,0,20,0,20,0,20
4,4,0,4,0,4,0,20,0,20,0,...,0,20,0,20,0,20,0,20,0,20


In [3]:
def get_reward(state):
    """
    The patient gets reward 1 in all states Stage k (k=1,2,...,18) and 
    reward 0.2 in the "Posttransplant Life
    
    Args: 
        state: current state
    
    Returns: 
        Reward associated with exiting state,
    """
    if state < 18:
        reward = 1.
    elif state == 18:
        reward = 0.2
    else:
        reward = 0.
    return reward 


def b(state, action):
    """
    Mapping of state and action to probability of action given state
    for behaviour policy. Transition probabilities taken from notebook.
    
    Args:
        state: current state
        action: current action
        
    Returns:
        Probability of action given state.
    """
    # placeholder probabilities of 0 added for post-transplant and death
    transplant_pr = [.02 for i in range(13)] + [.05, .1, .2, .4, .6, 0., 0.]
    return transplant_pr[state] if action == 1 else 1-transplant_pr[state]
    

def monte_carlo_control(data=elsd, gamma=0.97, T=49):
    """
    Implementation of `Off policy for MC control for estimating π \sim \pi_*` 
    algorithm from Reinforcement Learning Book (page 111),
    
    Args:
        data: pandas dataframe containing sequences as rows.
        gamma: discount parameter
        T: Length of episodes to consider
        
    Returns:
        π: Estimate of optimal policy $\pi_*$
        q: State Action Value matrix
    """
    # 20 states, 2 actions
    C = np.zeros((20, 2))
    Q = np.zeros((20, 2))
    
    # Greedy policy of target policy
    π = np.argmax(Q, axis=1)

    for index, episode in data.iterrows():
        # States and actions are concurrent, extracting into 
        # separate lists
        states = episode[::2]
        actions = episode[1::2]
        
        G = 0 # Sum of discounted rewards
        W = 1 # Importance sampling ratio weights
        
        # Traversing backward through timestep T-1 to t
        for t in reversed(range(0,T)):
            # Only update from first death record
            #if states[t-1] == 20:
            #    continue
            
            # 0-indexing states for easier use with arrays
            S, A = states[t]-1, actions[t]
            R = get_reward(S)
            G = gamma*G + R  
            C[S, A] += W
            Q[S, A] += (W / C[S, A]) * (G - Q[S, A])
            
            π[S] = np.argmax(Q[S])

            if A !=  π[S]:
                break
            W *= (1 / b(S, A)) 
    return π, Q

policy, q = monte_carlo_control()

In [4]:
policy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0])

From our estimate of $\pi_*$ we should not transplant unless we are in stage 17 or 18 of the disease.